# Imports

In [1]:
import pyAgrum as gum
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)

# Read and Make Classifiers Ready

In [2]:
df = pd.read_csv("newOutputs/classifiers1.csv", index_col=0)

def normalize(col):
    # TODO: GROTER DAN WAT????
    if max(col) > 1:
        return col/max(col)
    return col

columns = ["items","nodes","timesteps","arcs_ratio","values","PC1","PC2","PC3","PC4","PCweight1","PCweight2","PCweight3","PCweight4","PCAsAbove95","maxdegree/nodes","roots/nodes","leaves/nodes","longestpath/nodes","mean_constraint","mean_nr_similarities/nodes","mean_degree","nodestates","arcs","items/nodestates","items/arcs"]
df[columns] = df[columns].apply(normalize)

In [3]:
def combine(row):
    return row["algorithm"]+"_"+row["scorefunction"]

df["algorithmMetric"] = df.apply(combine, axis=1)
df = df.drop(["algorithm","scorefunction"], axis=1)
df

,items,nodes,timesteps,arcs_ratio,values,fscore,tp,fp,tn,fn,structure_id,PC1,PC2,PC3,PC4,PCweight1,PCweight2,PCweight3,PCweight4,PCAsAbove95,maxdegree/nodes,roots/nodes,leaves/nodes,longestpath/nodes,communities/nodes,mean_constraint,mean_efficiency,s_metric/arcs,mean_similarity,mean_nr_similarities/nodes,mean_degree,PHI,nodestates,arcs,items/nodestates,items/arcs,hamming,items/PHI,algorithmMetric
0,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0.026564,0.019496,0.010658,2.642740e-35,0.468358,0.343737,0.187905,4.659432e-34,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.666667,9.090909,HC_BDEU
1,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0.029963,0.021785,0.010271,4.171819e-36,0.483127,0.351269,0.165604,6.726657e-35,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.666667,9.090909,HC_AIC
2,0.01,0.50,0.333333,0.666666,0.5,0.500000,1,0,9,2,0,0.026530,0.014650,0.008163,3.406339e-35,0.537660,0.296901,0.165439,6.903346e-34,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.833333,9.090909,HC_BIC
3,0.01,0.50,0.333333,0.666666,0.5,0.400000,1,1,8,2,0,0.022533,0.020020,0.009400,3.650480e-34,0.433719,0.385348,0.180933,7.026453e-33,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.750000,9.090909,HC_K2
4,0.01,0.50,0.333333,0.666666,0.5,0.571429,2,2,7,1,0,0.025684,0.022297,0.013040,2.438681e-35,0.420904,0.365403,0.213693,3.996497e-34,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.750000,9.090909,HC_L2L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94495,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,0.012711,0.008655,0.003057,1.079689e-03,0.485668,0.330718,0.116812,4.125467e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,0.933333,833.333333,MMHC_BDEU
94496,1.00,0.75,0.333333,0.666666,0.5,1.000000,5,0,25,0,9,0.012960,0.008830,0.003337,1.185440e-03,0.479771,0.326879,0.123521,4.388570e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,1.000000,833.333333,MMHC_BIC
94497,1.00,0.75,0.333333,0.666666,0.5,0.600000,3,2,23,2,9,0.013470,0.008317,0.003217,1.127278e-03,0.502318,0.310161,0.119979,4.203714e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,0.866667,833.333333,MMHC_K2
94498,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,0.012971,0.008728,0.003197,1.183764e-03,0.484004,0.325684,0.119298,4.417248e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,0.933333,833.333333,3OFF2_Constraint


In [4]:
dfhot = pd.get_dummies(df, columns=["algorithmMetric"], dtype=int)
dfhot

,items,nodes,timesteps,arcs_ratio,values,fscore,tp,fp,tn,fn,structure_id,PC1,PC2,PC3,PC4,PCweight1,PCweight2,PCweight3,PCweight4,PCAsAbove95,maxdegree/nodes,roots/nodes,leaves/nodes,longestpath/nodes,communities/nodes,mean_constraint,mean_efficiency,s_metric/arcs,mean_similarity,mean_nr_similarities/nodes,mean_degree,PHI,nodestates,arcs,items/nodestates,items/arcs,hamming,items/PHI,algorithmMetric_3OFF2_Constraint,algorithmMetric_HC_AIC,algorithmMetric_HC_BDEU,algorithmMetric_HC_BIC,algorithmMetric_HC_K2,algorithmMetric_HC_L2L,algorithmMetric_MIIC_Constraint,algorithmMetric_MMHC_BDEU,algorithmMetric_MMHC_BIC,algorithmMetric_MMHC_K2,algorithmMetric_TABU_AIC,algorithmMetric_TABU_BDEU,algorithmMetric_TABU_BIC,algorithmMetric_TABU_K2,algorithmMetric_TABU_L2L
0,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0.026564,0.019496,0.010658,2.642740e-35,0.468358,0.343737,0.187905,4.659432e-34,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.666667,9.090909,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0.029963,0.021785,0.010271,4.171819e-36,0.483127,0.351269,0.165604,6.726657e-35,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.666667,9.090909,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.01,0.50,0.333333,0.666666,0.5,0.500000,1,0,9,2,0,0.026530,0.014650,0.008163,3.406339e-35,0.537660,0.296901,0.165439,6.903346e-34,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.833333,9.090909,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0.01,0.50,0.333333,0.666666,0.5,0.400000,1,1,8,2,0,0.022533,0.020020,0.009400,3.650480e-34,0.433719,0.385348,0.180933,7.026453e-33,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.750000,9.090909,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0.01,0.50,0.333333,0.666666,0.5,0.571429,2,2,7,1,0,0.025684,0.022297,0.013040,2.438681e-35,0.420904,0.365403,0.213693,3.996497e-34,0.750000,0.750000,0.25,0.750000,0.5,0.75,0.083333,0.250000,3.0,0.587500,0.625000,0.187500,11,0.000244,0.250000,0.01,0.01,0.750000,9.090909,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94495,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,0.012711,0.008655,0.003057,1.079689e-03,0.485668,0.330718,0.116812,4.125467e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,0.933333,833.333333,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
94496,1.00,0.75,0.333333,0.666666,0.5,1.000000,5,0,25,0,9,0.012960,0.008830,0.003337,1.185440e-03,0.479771,0.326879,0.123521,4.388570e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,1.000000,833.333333,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
94497,1.00,0.75,0.333333,0.666666,0.5,0.600000,3,2,23,2,9,0.013470,0.008317,0.003217,1.127278e-03,0.502318,0.310161,0.119979,4.203714e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,0.866667,833.333333,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
94498,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,0.012971,0.008728,0.003197,1.183764e-03,0.484004,0.325684,0.119298,4.417248e-02,0.666667,0.333333,0.50,0.333333,0.5,0.50,0.500000,0.666667,3.8,0.183333,0.222222,0.138889,12,0.000977,0.416667,0.25,0.60,0.933333,833.333333,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Prediction

In [5]:
%%time
columns = ['items', #'arcs_ratio', 
   'PC1',
    'PCweight1', 
    'PCAsAbove95', 
    'maxdegree/nodes',
   'leaves/nodes', 'roots/nodes',  
   'mean_constraint', 
    'mean_degree',
   'items/nodestates', 'items/arcs',
    'PHI', 'items/PHI',
   'algorithmMetric_3OFF2_Constraint', 'algorithmMetric_HC_AIC',
   'algorithmMetric_HC_BDEU', 'algorithmMetric_HC_BIC',
   'algorithmMetric_HC_K2', 'algorithmMetric_HC_L2L',
   'algorithmMetric_MIIC_Constraint', 'algorithmMetric_MMHC_BDEU',
   'algorithmMetric_MMHC_BIC', 'algorithmMetric_MMHC_K2',
   'algorithmMetric_TABU_AIC', 'algorithmMetric_TABU_BDEU',
   'algorithmMetric_TABU_BIC', 'algorithmMetric_TABU_K2',
   'algorithmMetric_TABU_L2L']

X_train, X_test, y_train, y_test = train_test_split(dfhot[columns], dfhot["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names_out(X_train.columns)), pd.DataFrame(poly.fit_transform(X_test), columns=poly.get_feature_names_out(X_test.columns))
regr = XGBRegressor(n_estimators=500, max_depth=14, eta=0.1, subsample=0.7, colsample_bytree=0.8)#, colsample_bynode=0.8)

regr.fit(X_train_poly, y_train)

print(regr.score(X_test_poly, y_test))

0.6211489969798686
CPU times: total: 47min 11s
Wall time: 4min 36s


# Feature Importance

In [6]:
def split(features):
    split1 = features.split(" ")
    if len(split1) > 1:
        return (split1[0], split1[1])
    
    split2 = features.split("^")
    if len(split2) > 1:
        return (split2[0], split2[0])
    
    return ('1',features)

features = pd.DataFrame({"features":regr.feature_names_in_, "importance":regr.feature_importances_}).sort_values("importance",ascending=True)
features["feature1"], features["feature2"] = zip(*features["features"].map(split))
features["importance"] *= 1 / max(features["importance"])

In [7]:
columns = ['1'] + list(X_train.columns)
combinations = pd.DataFrame()

for _, row in features.iterrows():
    if row["feature1"] == 2:
        print("A")
    combinations.loc[row["feature1"], row["feature2"]] = row["importance"]
    combinations.loc[row["feature2"], row["feature1"]] = row["importance"]
    
combinations.style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1)

,1,algorithmMetric_HC_BDEU,algorithmMetric_HC_BIC,algorithmMetric_HC_K2,algorithmMetric_HC_L2L,algorithmMetric_MIIC_Constraint,algorithmMetric_MMHC_BDEU,algorithmMetric_MMHC_BIC,algorithmMetric_MMHC_K2,algorithmMetric_TABU_AIC,algorithmMetric_TABU_BDEU,algorithmMetric_TABU_BIC,algorithmMetric_TABU_K2,algorithmMetric_TABU_L2L,algorithmMetric_HC_AIC,items/arcs,items/PHI,PHI,algorithmMetric_3OFF2_Constraint,leaves/nodes,items,items/nodestates,maxdegree/nodes,roots/nodes,PC1,PCAsAbove95,PCweight1,mean_constraint,mean_degree
1,0.000000,0.011028,0.010704,0.011680,0.011809,0.073960,0.019516,0.019328,0.016946,0.016587,0.010991,0.016290,0.012170,0.015360,0.010914,0.008434,0.122096,0.015052,0.059728,0.006450,0.033470,0.004283,0.006767,0.007448,0.007681,0.010654,0.011076,0.022492,0.014435
algorithmMetric_HC_BDEU,0.011028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068205,0.043744,0.064518,0.000000,0.062205,0.015300,0.023393,0.056878,0.068771,0.037351,0.043371,0.049327,0.058434,0.036592
algorithmMetric_HC_BIC,0.010704,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044574,0.121952,0.050071,0.000000,0.046507,0.011677,0.018309,0.052467,0.062509,0.041922,0.094500,0.045608,0.059179,0.054757
algorithmMetric_HC_K2,0.011680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052290,0.049910,0.061314,0.000000,0.065303,0.016603,0.028402,0.070892,0.081378,0.030664,0.052161,0.046461,0.040651,0.085734
algorithmMetric_HC_L2L,0.011809,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067852,0.027570,0.046953,0.000000,0.045211,0.016303,0.016761,0.074235,0.067482,0.038184,0.030774,0.042920,0.039356,0.096242
algorithmMetric_MIIC_Constraint,0.073960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075851,0.049664,0.290435,0.000000,0.746196,0.035365,0.025314,0.390620,0.098556,0.072300,0.094213,0.063016,0.312838,0.232382
algorithmMetric_MMHC_BDEU,0.019516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034487,0.053446,0.918858,0.000000,0.114041,0.011675,0.025304,0.259287,0.566074,0.048103,0.124028,0.080169,0.755627,0.030048
algorithmMetric_MMHC_BIC,0.019328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044865,0.042459,0.548425,0.000000,0.028518,0.026910,0.018315,0.080476,0.086573,0.073911,0.637982,0.045508,0.305011,0.035895
algorithmMetric_MMHC_K2,0.016946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063073,0.099345,1.000000,0.000000,0.053587,0.029387,0.047317,0.047035,0.310817,0.049319,0.403332,0.061022,0.425726,0.154473
algorithmMetric_TABU_AIC,0.016587,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085161,0.047291,0.076874,0.000000,0.049156,0.017964,0.033422,0.055658,0.047141,0.044844,0.044482,0.044548,0.056107,0.027502
